In [1]:
from traditional_model import *
import time

This nodebook is to run and store the results of the best classification model from the previous assignment.
The best obtained model is using a RandomForest classifier with no feature selection and robust scaling. Finally, a holdout approach is used to compute the resulting metric.


## Load and prepare the data
The wine_quality and the bank_marketing data has an inbalanced target variable, therefore SMOTE is used to get a balanced target column.


In [2]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [3]:
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
cong_voting.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa,class
0,140,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,383,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1
2,201,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,297,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0
4,309,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


In [4]:
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)
bank_marketing.head()

,age,default,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,...,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,poutcome_failure,poutcome_nonexistent,poutcome_success,class
0,56,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
1,57,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
2,37,0.0,1.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0
3,40,0.0,0.0,0.0,1,999,0,1.1,93.994,-36.4,...,0,0,0,0,0,0,0,1,0,0
4,56,0.0,0.0,1.0,1,999,0,1.1,93.994,-36.4,...,0,1,0,0,0,0,0,1,0,0


In [5]:
results = pd.DataFrame(columns=["method", 'dataset','Average Accuracy','Average Training Time'])
datasets = {
    'wine_quality': wine_quality,
    'congression_voting': cong_voting,
    'bank_marketing': bank_marketing
}
for dataset_name, dataset in datasets.items():
    """ if dataset_name == 'wine_quality' or dataset_name == 'bank_marketing':
        target_column = 'class'
        X,Y = apply_smote(dataset,target_column)
    else:
        X = dataset.drop(columns=["class"])
        Y = dataset["class"] """
    X = dataset.drop(columns=["class"])
    Y = dataset["class"]
    
    tmp_res = evaluate_model(X,Y)
    lineToAppend = {"method" : "RandomForest", "dataset": dataset_name, "Average Accuracy": tmp_res["f1"], "Average Training Time": tmp_res["training_time"]}
    results = results.append(lineToAppend, ignore_index=True)
results

c:\Users\zsomb\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zsomb\AppData\Local\Temp\ipykernel_33512\3000338022.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(lineToAppend, ignore_index=True)
C:\Users\zsomb\AppData\Local\Temp\ipykernel_33512\3000338022.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(lineToAppend, ignore_index=True)
C:\Users\zsomb\AppData\Local\Temp\ipykernel_33512\3000338022.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

,method,dataset,Average Accuracy,Average Training Time
0,RandomForest,wine_quality,0.657637,0.793370
1,RandomForest,congression_voting,0.954380,0.104006
2,RandomForest,bank_marketing,0.874207,2.055958


In [7]:
results.to_csv('./results/traditional_results.csv', index=False)